In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
import PIL
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [6]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [7]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images('./data/'))
data = []
labels = []

for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)
data = np.array(data, dtype="float32")
labels = np.array(labels)

[INFO] loading images...


In [8]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)

aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [9]:
baseModel = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))

In [10]:
baseModel.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [12]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [13]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [14]:
for layer in baseModel.layers:
    layer.trainable = False

In [15]:
opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
model.compile(optimizer=opt, metrics=['accuracy'], loss='binary_crossentropy')

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [17]:
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

  ...
    to  
  ['...']
Train for 34 steps, validate on 276 samples
Epoch 1/20
34/34 [==============================] - 24s 704ms/step - loss: 0.6211 - accuracy: 0.6470 - val_loss: 0.4599 - val_accuracy: 0.7617
Epoch 2/20
34/34 [==============================] - 12s 359ms/step - loss: 0.4581 - accuracy: 0.8118 - val_loss: 0.2492 - val_accuracy: 0.9180
Epoch 3/20
34/34 [==============================] - 14s 410ms/step - loss: 0.3411 - accuracy: 0.8745 - val_loss: 0.1505 - val_accuracy: 0.9492
Epoch 4/20
34/34 [==============================] - 13s 374ms/step - loss: 0.3002 - accuracy: 0.8783 - val_loss: 0.1391 - val_accuracy: 0.9492
Epoch 5/20
34/34 [==============================] - 12s 363ms/step - loss: 0.2702 - accuracy: 0.9017 - val_loss: 0.0641 - val_accuracy: 0.9766
Epoch 6/20
34/34 [==============================] - 17s 495ms/step - loss: 0.2274 - accuracy: 0.9195 - val_loss: 0.0566 - val_accuracy: 0.9844
Epoch 7/20
34/34 [==============================] - 14s 414ms/step - loss

In [33]:
image = load_img('./download (1).jpg', target_size=(224, 224))      ############### Add in the path of the image you want to test h
image = img_to_array(image)
image = preprocess_input(image)

In [34]:
image = np.expand_dims(image, 0)

In [35]:
image.shape

(1, 224, 224, 3)

In [36]:
res = model.predict(image)

In [38]:
res[0][0]

0.99804306